In [1]:
# Here are some libraries that we'll need for this process
import pandas as pd
from datetime import datetime
# import pyodbc as dbapi
from tqdm import tqdm
from IPython.core.display import Image
from IPython import display
import requests
import os.path
from datetime import timedelta
import json
import pprint
from collections import defaultdict
import re
import numpy as np
import matplotlib.pyplot as plt
import csv

pp = pprint.PrettyPrinter(indent=4)
import snipeit
# from snipeit import Models

server='https://sawpit.app'
token='yourtokenhere'


In [2]:
#load first sheet of excel into a dataframe
df = pd.read_excel('Master Inventory - RMJC 4-15-2023.xlsx', index_col=0, sheet_name=0) 
#extract the unique values of Summary Name which will become the unique models 



In [3]:
#show first row of xl spreadsheet
for index,row in df.iterrows():
    print(index,row)
    break

RMT 221001 0013 SRL# NEW                         NaN
Inventory                        RMT
Supplier                      Delson
PO#                              NaN
Order #                        12372
Date             2022-10-01 00:00:00
Type                    Heavy Timber
QTY                              1.0
Width                            6.0
Height                           6.0
Length                          10.0
BDF                             30.0
Species                   POC - FOHC
Grade              #2/Btr Appearance
Dryness                        Green
Dryness Today                    NaN
Summary Name     POC - FOHC - 6x6x10
Condition                        NaN
BDF Cost                       2.743
Freight Cost                    0.52
Landed Cost                    3.263
Value                          97.89
Markup                          1.25
Sell                        122.3625
Age                               10
Status                     Available
Date Used             

In [4]:
def summaryToModel(s):
    
    # we split this string into   "POC - FOHC - 6x6x10"
    # print(s)
    #remove whitespace
    s = s.split('-')
    species = s[0].strip()
    boxedHeart = s[1].strip()
    dims = ''
    if len(s) > 2:
        dims = s[2].strip()
    summary = ' - '.join([species,boxedHeart,dims])
    # category 2 is heavy timber
    manufacturer_id = 1 # 1 is Unknown
    if species ==  "DF":
        manufacturer_id = 2
    if species ==  "POC":
        manufacturer_id = 3
    if species ==  "AYC":
        manufacturer_id = 4
    if species ==  "WRC":
        manufacturer_id = 5
    if species ==  "WO":
        manufacturer_id = 6
    d = {"name":summary, "category_id":2, "manufacturer_id":manufacturer_id, "fieldset_id":2}
    return d


#get a list of unique models
uniqueModels = list(map(summaryToModel, df["Summary Name"].unique().tolist()))
print(len(uniqueModels))
pp.pprint(uniqueModels[0])

68
{   'category_id': 2,
    'fieldset_id': 2,
    'manufacturer_id': 3,
    'name': 'POC - FOHC - 6x6x10'}


In [5]:
#upload the unique models to sawpit
uploadModels = False
if uploadModels:
    M = snipeit.Models()
    sumRegex = re.compile(r"(\w+) - (\w+) - ([0123456789.x]+)")
    for m in uniqueModels:
        summaryMatch = sumRegex.match(m['name'])
        if(summaryMatch is None):
            #invalid name, skip it
            continue
        r = M.create(server, token,  json.dumps(m))
        if("error" in r):
            print(r)


In [6]:
#load a dict that maps name to model ids
M = snipeit.Models()
modelsDict = json.loads( M.get(server, token, 5000) )# Using default limit of 5000 for results
# pp.pprint(r)

# pp.pprint(modelsDict['rows'][1])
model_ids = defaultdict() #default of None
for row in modelsDict['rows']:
    model_ids[row['name']]=row['id']

model_ids

defaultdict(None,
            {'POC - FOHC - 6x6x10': 4,
             'POC - FOHC - 6x6x8': 5,
             'DF - FOHC - 4x10x14': 41,
             'POC - FOHC - 6x6x12': 6,
             'WRC - BHC - 6x8x6': 7,
             'WRC - BHC - 8x8x10': 8,
             'WRC - BHC - 8x12x16': 9,
             'DF - FOHC - 8x11x10': 10,
             'DF - FOHC - 12.5x12.5x8': 11,
             'DF - FOHC - 7.75x11.75x12': 12,
             'DF - FOHC - 7x16x22': 13,
             'DF - FOHC - 7.5x13.5x22': 14,
             'DF - FOHC - 7.5x15.5x18': 15,
             'DF - FOHC - 3.5x7.5x18': 16,
             'DF - FOHC - 3.5x11.5x16': 17,
             'DF - FOHC - 6x6x12': 18,
             'DF - FOHC - 6x8x16': 19,
             'DF - FOHC - 6x8x20': 20,
             'DF - FOHC - 8x8x20': 21,
             'DF - FOHC - 8x10x20': 22,
             'DF - FOHC - 8x8x16': 23,
             'DF - FOHC - 8x12x16': 24,
             'DF - FOHC - 8x8x8': 25,
             'DF - FOHC - 6x8x10': 26,
             'D

In [7]:
#load a dict that maps company name to company id 
C = snipeit.Company()
companiesDict = json.loads( C.get(server, token) )# Using default limit of 5000 for results
# pp.pprint(r)

# pp.pprint(modelsDict['rows'][1])
company_ids = defaultdict() #default of None
for row in companiesDict['rows']:
    company_ids[row['name']]=row['id']

company_ids

defaultdict(None,
            {'Found, Missing Label': 8,
             'Drops &amp; Offcuts': 7,
             'Trillium Dell': 6,
             'TETW': 5,
             'COTF': 4,
             'Delson': 3,
             'RMJC': 2,
             'RMT': 1})

In [8]:
#load a dict that maps supplier name to supplier id 
# S = snipeit.Supplier()
# suppliersDict = json.loads( S.get(server, token) )# Using default limit of 5000 for results
# pp.pprint(r)

# pp.pprint(modelsDict['rows'][1])
supplier_ids = defaultdict(lambda: 8,{
    'Delson':1,
    'Bluelinx':2,
    'Rosboro':3,
    'Boulder Lumber':4,
    'Hull-Oakes':5,
    'Customer':6,
    'TETW':7,
    'Unknown':8,
}) 

supplier_ids['Delson']

1

In [14]:

def strip(s):
    if isinstance(s, str):
        s = s.strip()
        if '-' == s: # change dash into blank
            return None
        return s
    return s




def formatDate(s):
    if isinstance(s, str):
        m, d, y = s.strip().split('/')
        return y+'-'+m.rjust(2,'0')+'-'+d.rjust(2,'0')

    return s

def getDims(row):
    summary = row['Summary Name']
    s = summary.split('-')
    species = s[0].strip()
    boxedHeart = s[1].strip()
    dims = None
    if len(s) > 2:
        dims = s[2].strip()
    return dims

def csvToAsset(row):
    summary = row['Summary Name']
    status_id = 10 #10 is imported, but does not have new label yet
    if 'Used' in row['Status']:
        status_id = 9
    # print("supplier ",strip(row['Supplier']),supplier_ids[strip(row['Supplier'])])
    d = {"asset_tag":row['SRL_OLD'], 
        "name":getDims(row), #We set the name to just the dims to help with sorting 
        "serial":row['SRL_OLD'], 
        "status_id":status_id, 
        "model_id": model_ids[summary], 
        "company_id": company_ids[strip(row['Inventory'])], 
        "supplier_id": supplier_ids[strip(row['Supplier'])], 
        "location_id": 1, # 1 is unknown
        "archived": False,
        "warranty_months": None,
        "depreciate": False,
        "requestable": True,
        "rtd_location_id": None,
        "_snipeit_bdf_8": row['BDF'],
        "_snipeit_bdf_cost_10": row['BDF Cost'],
        "_snipeit_condition_9": row['Condition'],
        "_snipeit_date_used_14": row['Date Used'],
        "_snipeit_dryness_3": row['Dryness'],
        "_snipeit_dryness_today_13": row['Dryness Today'],
        "_snipeit_freight_11": row['Freight Cost'],
        "_snipeit_grade_2": row['Grade'],
        "_snipeit_height_5": row['Height'],
        "_snipeit_length_7": row['Length'],
        "_snipeit_markup_12": row['Markup'],
        "_snipeit_project_15": row['Project'],
        "_snipeit_width_4": row['Width'],
        "order_number": row['Order'],
        "book_value": row['Sell'],
        "purchase_cost": row['Value'],
        "purchase_date": formatDate(row['Date']),
    }

    cleanDict =  {k: strip(v) for k, v in d.items()}

    return cleanDict

def getID(tag):
    try:
        A = snipeit.Assets()
        return A.getID(server, token, tag)
    except:
        return None


def uploadAssetIfNotExist(asset,update=False):
    tag = asset['asset_tag']
    A = snipeit.Assets()
    existing_id = getID(tag)
    # pp.pprint(asset)
    try:
        if existing_id is None:
            r = A.create(server, token, json.dumps(asset))
            if 'error' in r:
                print(f'uploading {tag} status error')
                print(r)
                r = json.loads(r)
                print(f'{tag} error msg = ' + r["messages"])
            else:
                print(f'{tag} uploaded ok')
              
            # 
            # r = A.create(server, token, json.dumps(asset))
            # print(r)
        else:
            if(update):
                r = A.updateDevice(server, token, str(existing_id) , json.dumps(asset))
                if r:
                    print(f'{tag} updated ok')
            else:
                print(f'{tag} exists, skipping upload')
    except:
        print(f'Exception uploading {tag} ')


InventoryCSVfn = 'Master Inventory - RMJC 4-15-2023 - Inventory Master List.csv'
import csv
sumRegex = re.compile(r"(\w+) - (\w+) - ([0123456789.x]+)")
#TODO: strip whitespace from column names

skiprows = 0
with open(InventoryCSVfn, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    rowcount = 0
    for row in reader:
        rowcount+=1
        if rowcount < skiprows:
            continue

        if(len(row['SRL_OLD']) < 3):
            print(f"Row {rowcount} is missing SRL_OLD, skipping it")
            continue
        summaryMatch = sumRegex.match(row['Summary Name'])
        if(summaryMatch is None):
            print(f"Row {rowcount} has bad summary {row['Summary Name']}, skipping it")
            continue
        asset = csvToAsset(row)
        uploadAssetIfNotExist(asset)
        if rowcount > 1000:
            break

    


RMT 221001 0013 uploaded ok
RMT 221001 0014 uploaded ok
RMT 221001 0015 uploaded ok
RMT 221001 0016 uploaded ok
RMT 221001 0017 uploaded ok
RMT 221001 0018 uploaded ok
RMT 221001 0019 uploaded ok
RMT 221001 0020 uploaded ok
RMT 221001 0021 uploaded ok
RMT 221001 0022 uploaded ok
RMT 221001 0023 uploaded ok
RMT 221001 0024 uploaded ok
RMT 221001 0025 uploaded ok
RMT 221001 0026 uploaded ok
RMT 221001 0027 uploaded ok
RMT 221001 0028 uploaded ok
RMT 221001 0029 uploaded ok
RMT 221001 0030 uploaded ok
RMT 221001 0031 uploaded ok
RMT 221001 0032 uploaded ok
RMT 221001 0033 uploaded ok
RMT 221001 0034 uploaded ok
RMT 221001 0035 uploaded ok
RMT 221001 0036 uploaded ok
RMT 221001 0037 uploaded ok
RMT 221001 0038 uploaded ok
RMT 221001 0039 uploaded ok
RMT 221001 0040 uploaded ok
RMT 221001 0041 uploaded ok
RMT 221001 0042 uploaded ok
RMT 221001 0043 uploaded ok
RMT 221001 0044 uploaded ok
RMT 221001 0045 uploaded ok
RMT 221001 0046 uploaded ok
RMT 221001 0047 uploaded ok
RMT 221001 0048 uplo

In [10]:

A = snipeit.Assets()

a = json.loads(A.get(server, token, 5)) # With a limit of results
pp.pprint(a['rows'][3])


{   'age': '10 months ago',
    'alt_barcode': 'https://sawpit.app/uploads/barcodes/c128-rmt-221001-0016.png',
    'asset_eol_date': None,
    'asset_tag': 'RMT 221001 0016',
    'assigned_to': None,
    'available_actions': {   'checkin': True,
                             'checkout': True,
                             'clone': True,
                             'delete': True,
                             'restore': False,
                             'update': True},
    'book_value': None,
    'byod': False,
    'category': {'id': 2, 'name': 'Heavy Timber'},
    'checkin_counter': 0,
    'checkout_counter': 0,
    'company': {'id': 1, 'name': 'RMT'},
    'created_at': {   'datetime': '2023-08-17 03:33:43',
                      'formatted': '08/17/2023 3:33AM'},
    'custom_fields': {   'BDF': {   'element': 'text',
                                    'field': '_snipeit_bdf_8',
                                    'field_format': 'NUMERIC',
                                    'value